In [110]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader


#loading pdfs
# LEGACY CODE
print("Loading PDFs")
# print("Course description...", end="\r", flush=True)
# pdf_loader = PyPDFLoader("rag_data/Umich_FA2024_course_description.pdf")
# course_description_text = pdf_loader.load()
# print("Loaded course description")
# print("Degree requirements...", end="\r", flush=True)
# pdf_loader = PyPDFLoader("rag_data/Umich_FA2024_LSA_degree_requ.pdf")
# degree_requirements_text = pdf_loader.load()
# print("Loaded degree requirements")
# print("Major minor description...", end="\r", flush=True)
# pdf_loader = PyPDFLoader("rag_data/Umich_FA2024_major_minor_description.pdf")
# major_minor_description_text = pdf_loader.load()
# print("Loaded major minor description")

# print("Subject mapping...", end="\r", flush=True)
# pdf_loader = PyPDFLoader("rag_data/Umich_FA2024_subject_mapping.pdf")
# subject_mapping_text = pdf_loader.load()
# print("Loaded subject mapping")

Loading PDFs
Loaded subject mapping


In [198]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# load course descriptions

loader = UnstructuredMarkdownLoader("rag_data/Umich_FA2024_course_description.md", mode="elements")
course_description_text = loader.load()

for doc in course_description_text:
    try:
        doc.metadata = {'source': 'rag_data/Umich_FA2024_course_description.md', 'course': str(doc.metadata['emphasized_text_contents'][0])}
    except:
        print(doc.metadata)
        doc.metadata = {'source': 'rag_data/Umich_FA2024_course_description.md'}


In [208]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd
import numpy as np

df = pd.read_csv("rag_data/Umich_FA2024_course_schedule.csv")
# combind Subject	Catalog Nbr
df["Class Identifier"] = df["Subject"] + " " + df["Catalog Nbr"]
# create a list of dfs for each unique class identifier
dfs = [df[df["Class Identifier"] == class_identifier] for class_identifier in df["Class Identifier"].unique()]
# move 'Class Identifier' column to the front of each df
dfs = [pd.concat([df["Class Identifier"], df.drop(columns=["Class Identifier"])], axis=1) for df in dfs]
# drop  Subject	Catalog Nbr columns from each df
dfs = [df.drop(columns=["Subject", "Catalog Nbr"]) for df in dfs]
# drop 'Unnamed: 23' column from each df
dfs = [df.drop(columns=["Unnamed: 23"]) for df in dfs]
# add | between elements in each row of each df
dfs = [df.applymap(lambda x: str(x) + " | ") for df in dfs]



from langchain_core.documents.base import Document

course_schedule_text = []
for df in dfs:
    doc = Document(page_content=df.to_string(index=False, header=False), metadata={"source": "rag_data/Umich_FA2024_course_schedule.csv", "course": df["Class Identifier"].iloc[0]})
    if len(doc.page_content) > 25000:
        # split the document into 4 smaller documents
        size = len(doc.page_content)
        for i in range(8):
            course_schedule_text.append(Document(page_content=doc.page_content[i*int(size/8):(i+1)*int(size/8)], metadata=doc.metadata))
    elif len(doc.page_content) > 10000:
            # split the document into 4 smaller documents
            size = len(doc.page_content)
            for i in range(4):
                course_schedule_text.append(Document(page_content=doc.page_content[i*int(size/4):(i+1)*int(size/4)], metadata=doc.metadata))

    elif len(doc.page_content) > 5000:
        # split the document into 2 smaller documents
        size = len(doc.page_content)
        for i in range(2):
            course_schedule_text.append(Document(page_content=doc.page_content[i*int(size/2):(i+1)*int(size/2)], metadata=doc.metadata))
    else:
        course_schedule_text.append(doc)


# number of documents with over 4k characters
course_schedule_text = [doc for doc in course_schedule_text if len(doc.page_content) < 5500]

/var/folders/5t/bmhkrbcn4yzgbdrrgwk7ps_r0000gn/T/ipykernel_4111/3119957408.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfs = [df.applymap(lambda x: str(x) + " | ") for df in dfs]


In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# chunking text
print("Chunking text...")
# LEGACY CODE
# pages = [course_description_text, degree_requirements_text, major_minor_description_text, subject_mapping_text, course_schedule_text]

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
# course_description_chunks = text_splitter.split_documents(course_description_text)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
# degree_requirements_chunks = text_splitter.split_documents(degree_requirements_text)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
# major_minor_description_chunks = text_splitter.split_documents(major_minor_description_text)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=30)
# subject_mapping_chunks = text_splitter.split_documents(subject_mapping_text)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
# course_schedule_chunks = text_splitter.split_documents(course_schedule_text)


# chunked_pages = []
# for page in pages:
#     chunks = text_splitter.split_documents(page)
#     chunked_pages.append(chunks)

Chunking text...


In [206]:
combined_chunks = course_description_text + course_schedule_text

In [210]:
from langchain_chroma import Chroma
from langchain_voyageai import VoyageAIEmbeddings
import dotenv
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

# cell to load database - VERY EXPENSIVE

dotenv.load_dotenv()
new_client = chromadb.PersistentClient(path = "./chroma_db", tenant = DEFAULT_TENANT, database = DEFAULT_DATABASE, settings = Settings())

embeddings = VoyageAIEmbeddings(
    voyage_api_key=dotenv.get_key(dotenv_path= ".env", key_to_get = "VOYAGEAI_KEY") , model="voyage-large-2-instruct")

vectorstore = Chroma.from_documents(documents= combined_chunks , embedding=embeddings, collection_name="umich_fa2024", client=new_client)

batch size None
